# **Asociación Avance 2 Proyecto**

#### Integrantes
- *Santiago Cassiano Rozo*
- *Fabian Camilo Castro Lozano*
- *David Jose Leon Aroca*

Instalamos e importamos las librerias necesarias para el desarrollo de los métodos a priori y FP Growth:

In [1]:
!pip install efficient-apriori
!pip install fpgrowth_py

from efficient_apriori import apriori
from fpgrowth_py import fpgrowth

import time
import numpy as np
import pandas as pd
from google.colab import drive

##Alistamiento del dataset

Conectamos el notebook con el contenido de Drive:

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


##POR SIMPLE BINNING

El primer paso es cargar el dataset categorico discretizado por igual Frecuencia desde Drive:

In [3]:
# Cargar el conjunto de datos desde un archivo CSV
df = pd.read_csv("/content/gdrive/Shareddrives/Minería de datos/Proyecto/I. Dataset/categorico.csv", sep=",")


Eliminamos la columna de los indices (Unnamed 0)

In [4]:
df = df.drop(df.columns[0], axis=1)

A continuación se ve el DataFrame, pero la columna de los índices sigue apareciendo debido a una visualización de la variable df, aunque al momento de tratarlo ya no está dentro del DataFrame:

In [5]:
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,Adulthood,management,married,tertiary,no,normal,yes,no,cellular,5,may,short,rarely,half,rarely,failure,no
1,Adulthood,technician,single,secondary,no,normal,yes,no,cellular,5,may,very short,rarely,half,rarely,failure,no
2,Adulthood,entrepreneur,married,secondary,no,normal,yes,yes,cellular,5,may,very short,rarely,half,rarely,failure,no
3,Adulthood,blue-collar,married,secondary,no,normal,yes,no,cellular,5,may,very short,rarely,half,rarely,failure,no
4,Adulthood,blue-collar,single,secondary,no,normal,no,no,cellular,5,may,very short,rarely,half,rarely,failure,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41085,Youth,student,single,tertiary,no,normal,no,no,cellular,17,nov,short,rarely,half,rarely,failure,yes
41086,Youth,technician,single,secondary,no,normal,no,yes,cellular,17,nov,short,rarely,half,rarely,failure,yes
41087,Adulthood,technician,married,tertiary,no,normal,no,no,cellular,17,nov,extensive,rarely,half,rarely,failure,yes
41088,Third age,retired,divorced,primary,no,normal,no,no,cellular,17,nov,short,rarely,half,rarely,failure,yes


Para hallar reglas interesantes decidimos darle un tratamiento al dataset en dos partes, los clientes que adquieren el depósito a termino en un DataFrame y los que no lo adquieren en otro:

In [6]:
df_yes = df[df['y'] == 'yes']
df_no = df[df['y'] == 'no']

Visualizamos los respectivos DataFrames

DataFrame de los que adquieren el depósito a término:

In [7]:
df_yes

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
79,Adulthood,admin.,married,secondary,no,normal,yes,no,cellular,5,may,extensive,rarely,half,rarely,failure,yes
123,Adulthood,services,married,secondary,no,normal,yes,no,cellular,5,may,half,rarely,half,rarely,failure,yes
161,Adulthood,admin.,married,tertiary,no,normal,no,no,cellular,5,may,half,rarely,half,rarely,failure,yes
262,Adulthood,management,single,tertiary,no,normal,yes,yes,cellular,5,may,half,rarely,half,rarely,failure,yes
445,Adulthood,technician,married,secondary,no,normal,yes,no,cellular,6,may,half,rarely,half,rarely,failure,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41084,Adulthood,admin.,single,secondary,no,normal,no,no,cellular,17,nov,very short,rarely,half,rarely,failure,yes
41085,Youth,student,single,tertiary,no,normal,no,no,cellular,17,nov,short,rarely,half,rarely,failure,yes
41086,Youth,technician,single,secondary,no,normal,no,yes,cellular,17,nov,short,rarely,half,rarely,failure,yes
41087,Adulthood,technician,married,tertiary,no,normal,no,no,cellular,17,nov,extensive,rarely,half,rarely,failure,yes


Al conocer que la columna "y" tiene todos los datos con el mismo valor podemos prescindir de ella cómo sigue:

In [8]:
df_yes = df_yes.drop('y', axis= 1)

In [9]:
df_yes

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
79,Adulthood,admin.,married,secondary,no,normal,yes,no,cellular,5,may,extensive,rarely,half,rarely,failure
123,Adulthood,services,married,secondary,no,normal,yes,no,cellular,5,may,half,rarely,half,rarely,failure
161,Adulthood,admin.,married,tertiary,no,normal,no,no,cellular,5,may,half,rarely,half,rarely,failure
262,Adulthood,management,single,tertiary,no,normal,yes,yes,cellular,5,may,half,rarely,half,rarely,failure
445,Adulthood,technician,married,secondary,no,normal,yes,no,cellular,6,may,half,rarely,half,rarely,failure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41084,Adulthood,admin.,single,secondary,no,normal,no,no,cellular,17,nov,very short,rarely,half,rarely,failure
41085,Youth,student,single,tertiary,no,normal,no,no,cellular,17,nov,short,rarely,half,rarely,failure
41086,Youth,technician,single,secondary,no,normal,no,yes,cellular,17,nov,short,rarely,half,rarely,failure
41087,Adulthood,technician,married,tertiary,no,normal,no,no,cellular,17,nov,extensive,rarely,half,rarely,failure


DataFrame de los que no lo adquieren:

In [10]:
df_no

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,Adulthood,management,married,tertiary,no,normal,yes,no,cellular,5,may,short,rarely,half,rarely,failure,no
1,Adulthood,technician,single,secondary,no,normal,yes,no,cellular,5,may,very short,rarely,half,rarely,failure,no
2,Adulthood,entrepreneur,married,secondary,no,normal,yes,yes,cellular,5,may,very short,rarely,half,rarely,failure,no
3,Adulthood,blue-collar,married,secondary,no,normal,yes,no,cellular,5,may,very short,rarely,half,rarely,failure,no
4,Adulthood,blue-collar,single,secondary,no,normal,no,no,cellular,5,may,very short,rarely,half,rarely,failure,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41073,Adulthood,blue-collar,married,secondary,no,normal,no,no,cellular,16,nov,short,rarely,near,rarely,failure,no
41075,Youth,services,single,secondary,no,normal,no,no,cellular,16,nov,very short,rarely,near,half,failure,no
41079,Adulthood,management,married,tertiary,no,normal,yes,yes,cellular,16,nov,very short,rarely,half,half,failure,no
41082,Adulthood,management,married,tertiary,no,normal,no,no,cellular,16,nov,short,rarely,half,rarely,failure,no


Al conocer que la columna "y" tiene todos los datos con el mismo valor podemos prescindir de ella cómo sigue:

In [11]:
df_no = df_no.drop('y', axis= 1)

In [12]:
df_no

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,Adulthood,management,married,tertiary,no,normal,yes,no,cellular,5,may,short,rarely,half,rarely,failure
1,Adulthood,technician,single,secondary,no,normal,yes,no,cellular,5,may,very short,rarely,half,rarely,failure
2,Adulthood,entrepreneur,married,secondary,no,normal,yes,yes,cellular,5,may,very short,rarely,half,rarely,failure
3,Adulthood,blue-collar,married,secondary,no,normal,yes,no,cellular,5,may,very short,rarely,half,rarely,failure
4,Adulthood,blue-collar,single,secondary,no,normal,no,no,cellular,5,may,very short,rarely,half,rarely,failure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41073,Adulthood,blue-collar,married,secondary,no,normal,no,no,cellular,16,nov,short,rarely,near,rarely,failure
41075,Youth,services,single,secondary,no,normal,no,no,cellular,16,nov,very short,rarely,near,half,failure
41079,Adulthood,management,married,tertiary,no,normal,yes,yes,cellular,16,nov,very short,rarely,half,half,failure
41082,Adulthood,management,married,tertiary,no,normal,no,no,cellular,16,nov,short,rarely,half,rarely,failure


Para los algoritmos a priori y FP-Growth es necesario convertir nuestro DataFrame en una lista de tuplas, que se entenderán cómo transacciones realizadas.

A continuación realizamos la conversión:

In [13]:
tuplas_yes = []
for _, row in df_yes.iterrows():
    tupla = tuple(f"{valor}_{columna}" for columna, valor in row.iteritems())
    tuplas_yes.append(tupla)



<ipython-input-13-1a628e3a66dd>:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tupla = tuple(f"{valor}_{columna}" for columna, valor in row.iteritems())


Visualizamos las transacciones con respuesta positiva de los clientes contactados:

In [14]:
tuplas_yes

[('Adulthood_age',
  'admin._job',
  'married_marital',
  'secondary_education',
  'no_default',
  'normal_balance',
  'yes_housing',
  'no_loan',
  'cellular_contact',
  '5_day',
  'may_month',
  'extensive_duration',
  'rarely_campaign',
  'half_pdays',
  'rarely_previous',
  'failure_poutcome'),
 ('Adulthood_age',
  'services_job',
  'married_marital',
  'secondary_education',
  'no_default',
  'normal_balance',
  'yes_housing',
  'no_loan',
  'cellular_contact',
  '5_day',
  'may_month',
  'half_duration',
  'rarely_campaign',
  'half_pdays',
  'rarely_previous',
  'failure_poutcome'),
 ('Adulthood_age',
  'admin._job',
  'married_marital',
  'tertiary_education',
  'no_default',
  'normal_balance',
  'no_housing',
  'no_loan',
  'cellular_contact',
  '5_day',
  'may_month',
  'half_duration',
  'rarely_campaign',
  'half_pdays',
  'rarely_previous',
  'failure_poutcome'),
 ('Adulthood_age',
  'management_job',
  'single_marital',
  'tertiary_education',
  'no_default',
  'normal_b

Conversión a lista de tuplas para las transacciones con respuesta negativa:

In [15]:
tuplas_no = []
for _, row in df_no.iterrows():
    tupla = tuple(f"{valor}_{columna}" for columna, valor in row.iteritems())
    tuplas_no.append(tupla)



<ipython-input-15-2419f2cfb844>:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  tupla = tuple(f"{valor}_{columna}" for columna, valor in row.iteritems())


Visualizamos las transacciones con respuesta negativa de los clientes contactados:

In [16]:
tuplas_no

[('Adulthood_age',
  'management_job',
  'married_marital',
  'tertiary_education',
  'no_default',
  'normal_balance',
  'yes_housing',
  'no_loan',
  'cellular_contact',
  '5_day',
  'may_month',
  'short_duration',
  'rarely_campaign',
  'half_pdays',
  'rarely_previous',
  'failure_poutcome'),
 ('Adulthood_age',
  'technician_job',
  'single_marital',
  'secondary_education',
  'no_default',
  'normal_balance',
  'yes_housing',
  'no_loan',
  'cellular_contact',
  '5_day',
  'may_month',
  'very short_duration',
  'rarely_campaign',
  'half_pdays',
  'rarely_previous',
  'failure_poutcome'),
 ('Adulthood_age',
  'entrepreneur_job',
  'married_marital',
  'secondary_education',
  'no_default',
  'normal_balance',
  'yes_housing',
  'yes_loan',
  'cellular_contact',
  '5_day',
  'may_month',
  'very short_duration',
  'rarely_campaign',
  'half_pdays',
  'rarely_previous',
  'failure_poutcome'),
 ('Adulthood_age',
  'blue-collar_job',
  'married_marital',
  'secondary_education',
  '

##A priori

Para la aplicación del algoritmo A priori de la libreria efficient_apriori aplicamos la función "apriori()" para hallar los itemsets frecuentes y las reglas de asociación, se tienen en cuenta un soporte mínimo de 0.9 y una confianza mínima de 0.9.

Visualizamos los itemsets frecuentes con respuesta positiva:

In [17]:
itemsets_yes, rules_yes = apriori(tuplas_yes, min_support=0.9, min_confidence=0.9)

print("Itemsets frecuentes:")
for itemset, support in itemsets_yes.items():
    print(f"{itemset}: {support}\n")

Itemsets frecuentes:
1: {('no_default',): 4053, ('no_loan',): 3728, ('cellular_contact',): 3835, ('rarely_campaign',): 3959}

2: {('cellular_contact', 'no_default'): 3786, ('cellular_contact', 'rarely_campaign'): 3708, ('no_default', 'no_loan'): 3693, ('no_default', 'rarely_campaign'): 3911}



Visualizamos las reglas de asociación generadas para la lista de transacciones positivas:

In [18]:
rules_yes

[{no_default} -> {cellular_contact},
 {cellular_contact} -> {no_default},
 {rarely_campaign} -> {cellular_contact},
 {cellular_contact} -> {rarely_campaign},
 {no_loan} -> {no_default},
 {no_default} -> {no_loan},
 {rarely_campaign} -> {no_default},
 {no_default} -> {rarely_campaign}]

Visualizamos los itemsets frecuentes con respuesta negativa:

In [19]:
itemsets_no, rules_no = apriori(tuplas_no, min_support=0.9, min_confidence=0.9)

print("Itemsets frecuentes:")
for itemset, support in itemsets_no.items():
    print(f"{itemset}: {support}\n")

Itemsets frecuentes:
1: {('Adulthood_age',): 34550, ('no_default',): 36268, ('cellular_contact',): 34848, ('rarely_campaign',): 33871, ('half_pdays',): 33934, ('rarely_previous',): 35385, ('failure_poutcome',): 35527}

2: {('Adulthood_age', 'no_default'): 33877, ('cellular_contact', 'failure_poutcome'): 33495, ('cellular_contact', 'no_default'): 34152, ('cellular_contact', 'rarely_previous'): 33350, ('failure_poutcome', 'half_pdays'): 33390, ('failure_poutcome', 'no_default'): 34817, ('failure_poutcome', 'rarely_previous'): 34445, ('no_default', 'rarely_previous'): 34674}

3: {('failure_poutcome', 'no_default', 'rarely_previous'): 33743}



Visualizamos las reglas de asociación generadas para la lista de transacciones negativas:

In [20]:
rules_no

[{no_default} -> {Adulthood_age},
 {Adulthood_age} -> {no_default},
 {failure_poutcome} -> {cellular_contact},
 {cellular_contact} -> {failure_poutcome},
 {no_default} -> {cellular_contact},
 {cellular_contact} -> {no_default},
 {rarely_previous} -> {cellular_contact},
 {cellular_contact} -> {rarely_previous},
 {half_pdays} -> {failure_poutcome},
 {failure_poutcome} -> {half_pdays},
 {no_default} -> {failure_poutcome},
 {failure_poutcome} -> {no_default},
 {rarely_previous} -> {failure_poutcome},
 {failure_poutcome} -> {rarely_previous},
 {rarely_previous} -> {no_default},
 {no_default} -> {rarely_previous},
 {no_default, rarely_previous} -> {failure_poutcome},
 {failure_poutcome, rarely_previous} -> {no_default},
 {failure_poutcome, no_default} -> {rarely_previous},
 {rarely_previous} -> {failure_poutcome, no_default},
 {no_default} -> {failure_poutcome, rarely_previous},
 {failure_poutcome} -> {no_default, rarely_previous}]

Observamos la cantidad de reglas generadas para cada una de las listas de transacciones:

Reglas de las transacciones negativas:

In [21]:
len(rules_no)

22

Reglas de las transacciones positivas:

In [22]:
len(rules_yes)

8

Dentro de las reglas generadas para la lista de transacciones positivas es necesario identificar las que representan un patrón real, para eso seleccionamos las que tienen un lift mayor a 1 debido a que son las que presentan mayor correlación:

In [23]:
# Mostrar reglas de asociación fuertes con un lift mayor a 1.0
print("Reglas de asociación:")
for rule in rules_yes:
  if rule.lift > 1:
    print(rule)

Reglas de asociación:
{rarely_campaign} -> {cellular_contact} (conf: 0.937, supp: 0.904, lift: 1.002, conv: 1.027)
{cellular_contact} -> {rarely_campaign} (conf: 0.967, supp: 0.904, lift: 1.002, conv: 1.053)
{no_loan} -> {no_default} (conf: 0.991, supp: 0.900, lift: 1.003, conv: 1.272)
{no_default} -> {no_loan} (conf: 0.911, supp: 0.900, lift: 1.003, conv: 1.026)


Dentro de las reglas generadas para la lista de transacciones negativas es necesario identificar las que representan un patrón real, para eso seleccionamos las que tienen un lift mayor a 1 debido a que son las que presentan mayor correlación:

In [24]:
# Mostrar reglas de asociación fuertes con un lift mayor a 1.0
print("Reglas de asociación:")
for rule in rules_no:
  if rule.lift > 1:
    print(rule)

Reglas de asociación:
{failure_poutcome} -> {cellular_contact} (conf: 0.943, supp: 0.906, lift: 1.001, conv: 1.012)
{cellular_contact} -> {failure_poutcome} (conf: 0.961, supp: 0.906, lift: 1.001, conv: 1.017)
{rarely_previous} -> {cellular_contact} (conf: 0.942, supp: 0.902, lift: 1.000, conv: 1.006)
{cellular_contact} -> {rarely_previous} (conf: 0.957, supp: 0.902, lift: 1.000, conv: 1.008)
{half_pdays} -> {failure_poutcome} (conf: 0.984, supp: 0.903, lift: 1.024, conv: 2.464)
{failure_poutcome} -> {half_pdays} (conf: 0.940, supp: 0.903, lift: 1.024, conv: 1.373)
{rarely_previous} -> {failure_poutcome} (conf: 0.973, supp: 0.931, lift: 1.013, conv: 1.487)
{failure_poutcome} -> {rarely_previous} (conf: 0.970, supp: 0.931, lift: 1.013, conv: 1.423)
{no_default, rarely_previous} -> {failure_poutcome} (conf: 0.973, supp: 0.912, lift: 1.013, conv: 1.471)
{failure_poutcome, no_default} -> {rarely_previous} (conf: 0.969, supp: 0.912, lift: 1.013, conv: 1.405)
{rarely_previous} -> {failure_po

##FP GROWTH

Para hacer un análisis objetivo y más a profundidad se decide utilizar FP GROWTH cómo el segundo método de asociación para tratar nuestro conjunto de datos catégoricos.

En esta primera parte se hace la ejecución del algoritmo "fpgrowth" de la librería "fpgrowth_py" asignandole los mismos valores de soporte y confianza que se utilizaron en el algoritmo a priori, con sus respectivos tiempos de ejecución, para obtener el conjunto de itemsets frecuentes y las reglas generadas para la lista de transacciones positivas:

In [25]:
#Ejecucion del algoritmo fpgrowth
start_time_fpgrowth = time.time()
#Se colocan los argumentos de manera que el soporte minimo de los itemsets frecuentes sea 2 (se pone en porcentaje)
#Y se pone como confianza minima para las reglas que se generen de 0%
itemsets_fp_yes, rules_fp_yes = fpgrowth(tuplas_yes, minSupRatio=0.9, minConf=0.9)
end_time_fpgrowth = time.time()
total_time_fpgrowth = end_time_fpgrowth - start_time_fpgrowth

Para lograr visualizar los resultados obtenidos es necesario organizar el return de la función fpgrowth, para ello se crea un diccionario en el que se hará la asignación de una manera muy similar a la del algoritmo a priori y se imprimen los resultados:

In [26]:
#Imprimir los tiempos de ejecucion de ambos algoritmos
print("Tiempo de ejecución (en segundos) del algoritmo FP-Growth: " + str(total_time_fpgrowth))

#Diccionario que ayuda a organizar los resultados de FP-Growth para que se vean igual que los resultados de Apriori
diccionario_yes = dict()
#Creamos las llaves del diccionario, cada llave representa el tamaño del conjunto de items frecuente
#El valor de cada llave sera una lista
for itemset in itemsets_fp_yes:
  if len(itemset) not in diccionario_yes.keys():
    diccionario_yes[len(itemset)] = list()

#Llenamos diccionario de las llaves anteriormente creadas con cada uno de los
#itemsets del tamaño correspondiente
for itemset in itemsets_fp_yes:
  diccionario_yes[len(itemset)].append(sorted(itemset))

#Imprimimos los conjuntos frecuentes obtenidos por FP-Growth
print("\nConjuntos obtenidos con el algoritmo FP-Growth:")
for key in sorted(diccionario_yes):
  print("Conjuntos frecuentes de tamaño "+str(key)+":")
  print(sorted(diccionario_yes[key]))


Tiempo de ejecución (en segundos) del algoritmo FP-Growth: 0.20971441268920898

Conjuntos obtenidos con el algoritmo FP-Growth:
Conjuntos frecuentes de tamaño 1:
[['cellular_contact'], ['no_default'], ['no_loan'], ['rarely_campaign']]
Conjuntos frecuentes de tamaño 2:
[['cellular_contact', 'no_default'], ['cellular_contact', 'rarely_campaign'], ['no_default', 'no_loan'], ['no_default', 'rarely_campaign']]


En está sección se imprimen las reglas generadas para la lista de transacciones positivas:

In [27]:
print("La cantidad de reglas generadas por el algoritmo FP-Growth fue: " + str(len(rules_fp_yes)))
print("Las reglas generadas por el algortimo FP-Growth son: ", end="")
for rule in rules_fp_yes:
  print(str(rule[0]) + "->" + str(rule[1]), end=", \n")

La cantidad de reglas generadas por el algoritmo FP-Growth fue: 8
Las reglas generadas por el algortimo FP-Growth son: {'no_loan'}->{'no_default'}, 
{'no_default'}->{'no_loan'}, 
{'rarely_campaign'}->{'cellular_contact'}, 
{'cellular_contact'}->{'rarely_campaign'}, 
{'no_default'}->{'cellular_contact'}, 
{'cellular_contact'}->{'no_default'}, 
{'no_default'}->{'rarely_campaign'}, 
{'rarely_campaign'}->{'no_default'}, 


En esta segunda parte se hace la ejecución del algoritmo "fpgrowth" de la librería "fpgrowth_py" asignandole los mismos valores de soporte y confianza que se utilizaron en el algoritmo a priori, con sus respectivos tiempos de ejecución, para obtener el conjunto de itemsets frecuentes y las reglas generadas para la lista de transacciones negativas:

In [28]:
#Ejecucion del algoritmo fpgrowth
start_time_fpgrowth = time.time()
#Se colocan los argumentos de manera que el soporte minimo de los itemsets frecuentes sea 2 (se pone en porcentaje)
#Y se pone como confianza minima para las reglas que se generen de 0%
itemsets_fp_no, rules_fp_no = fpgrowth(tuplas_no, minSupRatio=0.9, minConf=0.9)
end_time_fpgrowth = time.time()
total_time_fpgrowth = end_time_fpgrowth - start_time_fpgrowth

Para lograr visualizar los resultados obtenidos es necesario organizar el return de la función fpgrowth, para ello se crea un diccionario en el que se hará la asignación de una manera muy similar a la del algoritmo a priori y se imprimen los resultados:

In [29]:
#Imprimir los tiempos de ejecucion de ambos algoritmos
print("Tiempo de ejecución (en segundos) del algoritmo FP-Growth: " + str(total_time_fpgrowth))

#Diccionario que ayuda a organizar los resultados de FP-Growth para que se vean igual que los resultados de Apriori
diccionario_no = dict()
#Creamos las llaves del diccionario, cada llave representa el tamaño del conjunto de items frecuente
#El valor de cada llave sera una lista
for itemset in itemsets_fp_no:
  if len(itemset) not in diccionario_no.keys():
    diccionario_no[len(itemset)] = list()

#Llenamos diccionario de las llaves anteriormente creadas con cada uno de los
#itemsets del tamaño correspondiente
for itemset in itemsets_fp_no:
  diccionario_no[len(itemset)].append(sorted(itemset))

#Imprimimos los conjuntos frecuentes obtenidos por FP-Growth
print("\nConjuntos obtenidos con el algoritmo FP-Growth:")
for key in sorted(diccionario_no):
  print("Conjuntos frecuentes de tamaño "+str(key)+":")
  print(sorted(diccionario_no[key]))


Tiempo de ejecución (en segundos) del algoritmo FP-Growth: 2.3009133338928223

Conjuntos obtenidos con el algoritmo FP-Growth:
Conjuntos frecuentes de tamaño 1:
[['Adulthood_age'], ['cellular_contact'], ['failure_poutcome'], ['half_pdays'], ['no_default'], ['rarely_campaign'], ['rarely_previous']]
Conjuntos frecuentes de tamaño 2:
[['Adulthood_age', 'no_default'], ['cellular_contact', 'failure_poutcome'], ['cellular_contact', 'no_default'], ['cellular_contact', 'rarely_previous'], ['failure_poutcome', 'half_pdays'], ['failure_poutcome', 'no_default'], ['failure_poutcome', 'rarely_previous'], ['no_default', 'rarely_previous']]
Conjuntos frecuentes de tamaño 3:
[['failure_poutcome', 'no_default', 'rarely_previous']]


En está sección se imprimen las reglas generadas para la lista de transacciones positivas:

In [30]:
print("La cantidad de reglas generadas por el algoritmo FP-Growth fue: " + str(len(rules_fp_no)))
print("Las reglas generadas por el algortimo FP-Growth son: ", end="")
for rule in rules_fp_no:
  print(str(rule[0]) + "->" + str(rule[1]), end=", \n")

La cantidad de reglas generadas por el algoritmo FP-Growth fue: 22
Las reglas generadas por el algortimo FP-Growth son: {'half_pdays'}->{'failure_poutcome'}, 
{'failure_poutcome'}->{'half_pdays'}, 
{'no_default'}->{'Adulthood_age'}, 
{'Adulthood_age'}->{'no_default'}, 
{'rarely_previous'}->{'cellular_contact'}, 
{'cellular_contact'}->{'rarely_previous'}, 
{'failure_poutcome'}->{'cellular_contact'}, 
{'cellular_contact'}->{'failure_poutcome'}, 
{'no_default'}->{'cellular_contact'}, 
{'cellular_contact'}->{'no_default'}, 
{'rarely_previous'}->{'failure_poutcome'}, 
{'failure_poutcome'}->{'rarely_previous'}, 
{'rarely_previous'}->{'failure_poutcome', 'no_default'}, 
{'failure_poutcome'}->{'rarely_previous', 'no_default'}, 
{'no_default'}->{'rarely_previous', 'failure_poutcome'}, 
{'rarely_previous', 'failure_poutcome'}->{'no_default'}, 
{'rarely_previous', 'no_default'}->{'failure_poutcome'}, 
{'failure_poutcome', 'no_default'}->{'rarely_previous'}, 
{'rarely_previous'}->{'no_default'}, 


Al observar que los itemsets frecuentes y las reglas generadas por ambos algoritmos son idénticas con los mismos parametros de soporte y confianza podemos concluir que la asociación resultante es objetiva y las reglas generadas con un soporte de 0.9 y una confianza de 0.9 pasarán a ser analizadas con el fin de determinar cuales de ellas son realmente útiles para nuestro propósito de determinar cuales rasgos y características de los clientes hacen que adquieran un depósito a término y cuales hacen que no lo adquieran, para ser tenidas en cuenta dentro de nuestro modelo predictivo.